In [1]:
# Import the modules of interest
import pandas as pd
import geopandas as gpd
import ee
from sklearn.metrics import r2_score
ee.Initialize()


In [17]:
compositeImg = ee.Image('The layer you need')

covarsToUse = compositeImg.bandNames().removeAll(['Pixel_Lat', 'Pixel_Long'])

composite = compositeImg.select(covarsToUse)
    
dataset = ee.FeatureCollection("The Feature Collection to train")
nema_sampled = composite.sampleRegions(dataset, None, 927.6624232772797, geometries = True)
totalSize = nema_sampled.size().getInfo()
print('Number of observations:', totalSize)

varToModel = 'Total_Number'


Number of observations: 1695


In [3]:
def FCDFconv(fc):
    features = fc.getInfo()['features']
    dictarray = []

    for f in features:
        dict = f['properties']
        dictarray.append(dict)

    df = pd.DataFrame(dictarray)
    
    return df


In [4]:
# Buffer size (in m)
buffer_size = 50000

nema_wBuffer = nema_sampled.map(lambda f: f.buffer(buffer_size))

def BLOOcv(f):
    testFC = ee.FeatureCollection(f)
    
#     trainFC = nema_sampled.filter(ee.Filter.bounds(testFC).Not())
    trainFC = nema_sampled.filter(ee.Filter.geometry(testFC).Not())
    
    classifier = ee.Classifier.smileRandomForest(
                        numberOfTrees=20,
                        variablesPerSplit=8,
                        minLeafPopulation=3,
                        bagFraction=0.632 # All hyperparameters can be changed according to the best model hyperparameter combinations. 
                        ).setOutputMode('REGRESSION')
    
    trainedClassifer = classifier.train(trainFC, varToModel, covarsToUse)
    
    classified = ee.FeatureCollection(testFC).classify(classifier= trainedClassifer,
                         outputName= 'predicted')
    
    predicted = classified.first().get('predicted')
    
    return f.set('predicted', predicted).copyProperties(f)


result = ee.FeatureCollection(nema_wBuffer.limit(10).map(BLOOcv))

predObs = result.select([varToModel, 'predicted'])



In [5]:
predobsDict = predObs.getInfo()['features']

result_dataframe = pd.DataFrame(predobsDict)

predObsDf = result_dataframe.drop(['type', 'geometry'],1).join(result_dataframe.properties.apply(pd.Series))
predObsDf[['id', varToModel, 'predicted']]


,id,Total_Number,predicted
0,0000000000000000004e_0,958.000000,262.141565
1,000000000000000000ef_0,174.000000,1000.662806
2,000000000000000000fa_0,591.000000,408.029619
3,00000000000000000230_0,9259.000000,800.178645
4,00000000000000000320_0,677.735962,1905.733237
5,00000000000000000362_0,4090.000000,4814.930155
6,00000000000000000375_0,5867.000000,6201.641551
7,00000000000000000383_0,5787.000000,4081.329196
8,00000000000000000395_0,10053.000000,6478.036546
9,00000000000000000399_0,3902.000000,4174.910192


In [6]:
print(predObsDf.shape)
print(predObsDf.head())
r2_score(predObsDf[varToModel], predObsDf['predicted'])


(10, 4)
                       id                                         properties  \
0  0000000000000000004e_0  {'Total_Number': 958, 'predicted': 262.1415650...   
1  000000000000000000ef_0  {'Total_Number': 174, 'predicted': 1000.662805...   
2  000000000000000000fa_0  {'Total_Number': 591, 'predicted': 408.0296187...   
3  00000000000000000230_0  {'Total_Number': 9259, 'predicted': 800.178644...   
4  00000000000000000320_0  {'Total_Number': 677.735962, 'predicted': 1905...   

   Total_Number    predicted  
0    958.000000   262.141565  
1    174.000000  1000.662806  
2    591.000000   408.029619  
3   9259.000000   800.178645  
4    677.735962  1905.733237  


0.2274769879515287

In [ ]:
# Blocked Leave One Out cross-validation function:
def BLOOcv(f):
    testFC = ee.FeatureCollection(f)
    
    trainFC = nema_sampled.filter(ee.Filter.geometry(testFC).Not())
    
    classifier = ee.Classifier.smileRandomForest(
                        numberOfTrees=20,
                        variablesPerSplit=8,
                        minLeafPopulation=3,
                        bagFraction=0.632
                        ).setOutputMode('REGRESSION')
    
    trainedClassifer = classifier.train(trainFC, varToModel, covarsToUse)
    
    classified = ee.FeatureCollection(testFC).classify(classifier= trainedClassifer,
                         outputName= 'predicted')
    
    predicted = classified.first().get('predicted')
    
    return f.set('predicted', predicted).copyProperties(f)

In [54]:
# Buffer size (in m)
buffer_size = 50000

# Add buffer to FC of sampled observations
nema_wBuffer = nema_sampled.map(lambda f: f.buffer(buffer_size))

# Initiate empty DF
predObsDf = pd.DataFrame()

# Map BLOOcv over FC, untill all features are completed
while predObsDf.shape[0] < totalSize:
    if not predObsDf.empty:
        toMap = nema_wBuffer.filter(ee.Filter.inList('system:index', list(predObsDf['id'])).Not())
    else:
        toMap = nema_wBuffer
    
    print(predObsDf.shape[0])
    
    if totalSize-predObsDf.shape[0] > 10:
        result = ee.FeatureCollection(toMap.limit(10).map(BLOOcv))
        
    else:
        result = ee.FeatureCollection(toMap.map(BLOOcv))

    predObs = result.select([varToModel, 'predicted'])

    predobsDict = predObs.getInfo()['features']

    result_dataframe = pd.DataFrame(predobsDict)

    toAppend = result_dataframe.drop(['type', 'geometry'],1).join(result_dataframe.properties.apply(pd.Series))
    toAppend[['id', varToModel, 'predicted']]
    
    predObsDf = predObsDf.append(toAppend)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580


KeyboardInterrupt: 

In [39]:
# Print information on outputted DF
print(predObsDf.shape)
print(predObsDf.head())
print('Coefficient of Determination (R^2):', r2_score(predObsDf[varToModel], predObsDf['predicted']))

(1690, 4)
                       id                                         properties  \
0  0000000000000000004e_0  {'Total_Number': 958, 'predicted': 262.1415650...   
1  000000000000000000ef_0  {'Total_Number': 174, 'predicted': 1000.662805...   
2  000000000000000000fa_0  {'Total_Number': 591, 'predicted': 408.0296187...   
3  00000000000000000230_0  {'Total_Number': 9259, 'predicted': 800.178644...   
4  00000000000000000320_0  {'Total_Number': 677.735962, 'predicted': 1905...   

   Total_Number    predicted  
0    958.000000   262.141565  
1    174.000000  1000.662806  
2    591.000000   408.029619  
3   9259.000000   800.178645  
4    677.735962  1905.733237  


0.22747698795152882

In [71]:
results = pd.DataFrame(columns = ['Buffer_size', 'R2'])

buffer_sizes = [100, 1000, 100000, 50000, 100000]
for i in buffer_sizes:
    buffer_size = i
    print('Buffer size:', buffer_size)

    # Add buffer to FC of sampled observations
    nema_wBuffer = nema_sampled.map(lambda f: f.buffer(buffer_size))

    # Initiate empty DF
    predObsDf = pd.DataFrame()

    # Map BLOOcv over FC, untill all features are completed
    while predObsDf.shape[0] < totalSize:
        if not predObsDf.empty:
            toMap = nema_wBuffer.filter(ee.Filter.inList('system:index', list(predObsDf['id'])).Not())
        else:
            toMap = nema_wBuffer

        if totalSize-predObsDf.shape[0] > 25:
            result = ee.FeatureCollection(toMap.limit(25).map(BLOOcv))

        else:
            result = ee.FeatureCollection(toMap.map(BLOOcv))

        predObs = result.select([varToModel, 'predicted'])

        predobsDict = predObs.getInfo()['features']

        result_dataframe = pd.DataFrame(predobsDict)

        toAppend = result_dataframe.drop(['type', 'geometry'],1).join(result_dataframe.properties.apply(pd.Series))
        toAppend[['id', varToModel, 'predicted']]

        predObsDf = predObsDf.append(toAppend)
        
        print('Completed features:', predObsDf.shape[0])
    
    predObsDf.to_csv(i+"predObs.csv")
    results = results.append({'Buffer_size': buffer_size, 'R2': r2_score(predObsDf[varToModel], predObsDf['predicted'])}, ignore_index=True)
results.to_csv("BLOOcv_final_R2_results.csv")


Buffer size: 100
Completed features: 25
Completed features: 50
Completed features: 75
Completed features: 100
Completed features: 125
Completed features: 150
Completed features: 175
Completed features: 200
Completed features: 225
Completed features: 250
Completed features: 275
Completed features: 300
Completed features: 325
Completed features: 350
Completed features: 375


KeyboardInterrupt: 

In [70]:
print(results)

   Buffer_size        R2
0          1.0  0.189473
1      50000.0  0.206456
2     100000.0  0.135587
3     500000.0 -0.116321
